<a href="https://colab.research.google.com/github/simonamador/EEG-Channel-Selection-Algorithm/blob/main/SequentialSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import mne
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.1 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Funciones de modelos de clasificacion MLPNN, CNN
#Se usan activaciones lineales para no perder informacion (activacion podria
#perder informacion que sea negativa)
def findaccuraciesMLPNN(n,X,y,ch):
  X_tr, X_ts, y_train, y_test = train_test_split(
      X[:,ch,:], y, test_size=0.2, random_state=42)
  X_train, X_test = np.empty(shape=(X_tr.shape[0],n,113,1)), np.empty(shape=(X_ts.shape[0],n,113,1))
  X_train[:,:,:,0], X_test[:,:,:,0] = X_tr, X_ts
  #Generar modelo de MPLNN. 
  tf.random.set_seed(42)
  modelMLPNN = models.Sequential([
      layers.Flatten(input_shape=(n,113,1)),
      layers.Dense(25, activation='linear'),
      layers.Dense(25, activation='linear'),
      layers.Dense(25, activation='linear'),
      layers.Dense(2, activation='softmax')
  ])

  modelMLPNN.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
  history = modelMLPNN.fit(X_train, y_train, epochs=2, 
                    validation_data=(X_test, y_test))
  test_loss, test_acc = modelMPLNN.evaluate(X_test,  y_test, verbose=2)
  return test_acc

def findaccuraciesCNN(n,X,y,ch):
  X_tr, X_ts, y_train, y_test = train_test_split(
      X[:,ch,:], y, test_size=0.2, random_state=42)
  X_train, X_test = np.empty(shape=(X_tr.shape[0],n,113,1)), np.empty(shape=(X_ts.shape[0],n,113,1))
  X_train[:,:,:,0], X_test[:,:,:,0] = X_tr, X_ts
  #Generar modelo de CNN inspirado en modelo de Llorente-Vidrio et al. 
  tf.random.set_seed(42)
  modelCNN = models.Sequential([
    layers.Conv2D(3, (n, n), activation='linear', input_shape=(n,113,1))
    layers.MaxPooling2D((1, 1))
    layers.Conv2D(3, (1, 1), activation='linear')
    layers.MaxPooling2D((1, 1))
    layers.Flatten()
    layers.Dense(25, activation='linear'),
    layers.Dense(25, activation='linear'),
    layers.Dense(25, activation='linear'),
    layers.Dense(2, activation='softmax')
  ])

  modelCNN.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])
  history = modelCNN.fit(X_train, y_train, epochs=2, 
                    validation_data=(X_test, y_test))
  test_loss, test_acc = modelCNN.evaluate(X_test,  y_test, verbose=2)
  return test_acc

def indexing(seq,item,max):
  locs = []
  start_at = -1
  i=0
  while i < item.size-1:
    if item[i] == item[i+1]:
      try: c = item[i] != item[i-1]
      except ValueError:
            pass
      if c == True:
        start_at = -1
      if i == item.size-1:
        loc = seq.index(item[i],start_at+1)
        locs.append(loc)
        i+=1
      else:
        for v in range(2):
          loc = seq.index(item[i],start_at+1)
          locs.append(loc)
          start_at = loc
        i+=2
    else:
      try: c = item[i] == item[i-1]
      except ValueError:
        pass
      if c == True:
        loc = seq.index(item[i],start_at+1)
        locs.append(loc)
      else:
        loc = seq.index(item[i])
        locs.append(loc)
      i +=1
  if i == item.size-1:
    if item[item.size-1]==item[item.size-2]:
      loc = seq.index(item[item.size-1],start_at+1)
      locs.append(loc)
    else:
      loc = seq.index(item[item.size-1])
      locs.append(loc)
  return locs

In [ ]:
#Se extraen archivos de data preprocesada de EEG
with np.load('/content/drive/MyDrive/Colab Notebooks/Datosfiltrados/SubjectsData.npz', allow_pickle=True) as data:
  X = data['X']
  y = data['y']
del data #Se elimina para que no ocupe RAM

In [ ]:
ynew=[]
xnew=[]
#Reduccion a dos clases
for s in range(109):
  if s in [87,91,99]:
    continue
  else:
    yline =[]
    xline =[]
    for i in range(y[s].size):
      if y[s][i] in [1,2]:
        yline.append(y[s][i]-1)
        xline.append(X[s][i])
    yline = np.array(yline)
    ynew.append(yline)
    xline = np.array(xline)
    xnew.append(xline)
X = xnew
y = ynew

#Normalizacion
scaler = MinMaxScaler(feature_range=(0, 1))
for i in range(len(X)):
  for c in range(64):
    X[i][:,c,:] = scaler.fit_transform(X[i][:,c,:])

del xline, yline, s, i, c, xnew, ynew, scaler

In [ ]:
for s in range(0,106):

  #Set 1 electrodo
  acc = []
  for x in range(64):
    acci = findaccuraciesCNN(1,X[s],y[s],[x]) #Funcion para encontrar la precision
    acc.append(acci)
  nacc = np.array(acc) #Se genera vector de precisiones
  C = nacc[np.argsort(nacc)] #Se ordena por precision de menor a mayor
  D = C[-10:] #D = 10 mejores precisiones
  Best_acc = D[-1] #Mejor precision
  Nr = acc.index(Best_acc) #Se obtiene del mejor canale
  Mi = indexing(acc,D,Best_acc) #Se obtiene el valor de los mejores canales
  del acc, nacc, acci, C

  #Set 2 electrodos
  
  #Se obtiene una combinacion de los mejores canales, sin canales repetidos
  Hk = np.array(np.meshgrid(Nr, Mi)).T.reshape(-1,2)
  Hk1 = []
  for i in range(Hk.shape[0]):        
      if len(set(Hk[i].astype(list))) == Hk[i].size:
        Hk1.append(Hk[i])
  acc2 = []
  Hk1 = [list(e) for e in (set(tuple(row) for row in Hk1))]

  #Se evaluan
  for i in range(len(Hk1)):
    Best_accuracy = findaccuraciesCNN(2,X[s],y[s],Hk1[i])
    if Best_accuracy > Best_acc:
      Best_acc = Best_accuracy
      Nr = Hk1[i]
    acc2.append(Best_accuracy)
  nacc2 = np.array(acc2)
  C2 = nacc2[np.argsort(nacc2)]
  D2 = C2[-5:]
  index = indexing(acc2,D2,Best_acc)
  Mi2 = np.array(Hk1)[index]

  del C2, nacc2, acc2, Hk1

#Los siguientes sets repiten el proceso de 2 electrodos
#Set de 3 electrodos
  Hk = np.array(np.meshgrid(np.array(Mi2)[:,0], np.array(Mi2)[:,1], Mi)).T.reshape(-1,3)
  Hk2 = []
  for i in range(Hk.shape[0]):
      if len(set(Hk[i].astype(list))) == Hk[i].size:
        Hk2.append(Hk[i])
  Hk2 = [list(e) for e in (set(tuple(row) for row in Hk2))]
  acc3 = []
  for i in range(len(Hk2)):
    Best_accuracy = findaccuraciesCNN(3,X[s],y[s],Hk2[i])
    if Best_accuracy > Best_acc:
      Best_acc = Best_accuracy
      Nr = Hk2[i]
    acc3.append(Best_accuracy)
  nacc3 = np.array(acc3)
  C3 = nacc3[np.argsort(nacc3)]
  D3 = C3[-5:]
  index = indexing(acc3,D3,Best_acc)
  Mi3 = np.array(Hk2)[index]

  del C3, nacc3, acc3, Hk2

  #Set de 4 electrodos
  Hk = np.array(np.meshgrid(np.array(Mi3)[:,0], np.array(Mi3)[:,1], np.array(Mi3)[:,2],Mi)).T.reshape(-1,4)
  Hk3 = []
  for i in range(Hk.shape[0]):
      if len(set(Hk[i].astype(list))) == Hk[i].size:
        Hk3.append(Hk[i])
  Hk3 = [list(e) for e in (set(tuple(row) for row in Hk3))]
  acc4 = []
  for i in range(len(Hk3)):
    Best_accuracy = findaccuraciesCNN(4,X[s],y[s],Hk3[i])
    if Best_accuracy > Best_acc:
      Best_acc = Best_accuracy
      Nr = Hk3[i]
    acc4.append(Best_accuracy)
  nacc4 = np.array(acc4)
  C4 = nacc4[np.argsort(nacc4)]
  D4 = C4[-5:]
  index = indexing(acc4,D4,Best_acc)
  Mi4 = np.array(Hk3)[index]

  del C4, nacc4, acc4, Hk3


  #Set de 5 electrodos
  Hk = np.array(np.meshgrid(np.array(Mi4)[:,0], np.array(Mi4)[:,1], np.array(Mi4)[:,2], np.array(Mi4)[:,3], Mi)).T.reshape(-1,5)
  Hk4 = []
  for i in range(Hk.shape[0]):
      if len(set(Hk[i].astype(list))) == Hk[i].size:
        Hk4.append(Hk[i])
  Hk4 = [list(e) for e in (set(tuple(row) for row in Hk4))]
  acc5 = []
  for i in range(len(Hk4)):
    Best_accuracy = findaccuraciesCNN(5,X[s],y[s],Hk4[i])
    if Best_accuracy > Best_acc:
      Best_acc = Best_accuracy
      Nr = Hk4[i]
    acc5.append(Best_accuracy)
  nacc5 = np.array(acc5)
  C5 = nacc5[np.argsort(nacc5)]
  D5 = C5[-5:]
  index = indexing(acc5,D5,Best_acc)
  Mi5 = np.array(Hk4)[index]

  del C5, nacc5, acc5, Hk4

  #Set de 6 electrodos
  Hk = np.array(np.meshgrid(np.array(Mi5)[:,0], np.array(Mi5)[:,1], np.array(Mi5)[:,2], np.array(Mi5)[:,3], np.array(Mi5)[:,4], Mi)).T.reshape(-1,6)
  Hk5 = []
  for i in range(Hk.shape[0]):
      if len(set(Hk[i].astype(list))) == Hk[i].size:
        Hk5.append(Hk[i])
  Hk5 = [list(e) for e in (set(tuple(row) for row in Hk5))]
  acc6 = []
  for i in range(len(Hk5)):
    Best_accuracy = findaccuraciesCNN(6,X[s],y[s],Hk5[i])
    if Best_accuracy > Best_acc:
      Best_acc = Best_accuracy
      Nr = Hk5[i]
    acc6.append(Best_accuracy)
  nacc6 = np.array(acc6)
  C6 = nacc6[np.argsort(nacc6)]
  D6 = C6[-5:]
  index = indexing(acc6,D6,Best_acc)
  Mi6 = np.array(Hk5)[index]

  del C6, nacc6, acc6, Hk5

  #Guardar datos

  np.savez_compressed('/content/drive/MyDrive/Colab Notebooks/Datosfiltrados/Subject'+str(s+1)+'.npz', 
                      D = np.array([D,D2,D3,D4,D5,D6], dtype=object), 
                      Mi = np.array([Mi,Mi2,Mi3,Mi4,Mi5,Mi6], dtype=object), 
                      NrAcc = np.array([Nr, Best_acc], dtype=object),
                      allow_pickle=True)
  del Best_acc, Best_accuracy, D, D2, D3, D4, D5, D6, Hk, Mi, Mi2, Mi3, Mi4, Mi5, Mi6, Nr

Se han truncado las últimas 5000 líneas del flujo de salida.
Total params: 1,760
Trainable params: 1,760
Non-trainable params: 0
_________________________________________________________________
5/5 [==============================] - 1s 60ms/step - loss: 0.6886 - accuracy: 0.8543 - val_loss: 0.6805 - val_accuracy: 0.8684
2/2 - 0s - loss: 0.6805 - accuracy: 0.8684 - 30ms/epoch - 15ms/step
Model: "sequential_3492"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6984 (Conv2D)        (None, 1, 108, 3)         111       
                                                                 
 max_pooling2d_6984 (MaxPool  (None, 1, 108, 3)        0         
 ing2D)                                                          
                                                                 
 conv2d_6985 (Conv2D)        (None, 1, 108, 3)         12        
                                                        